<a href="https://colab.research.google.com/github/nokdujeon83/20242R0136COSE47402/blob/main/FinalProject/DL_final_project_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

Install OpenAI Python Package

In [1]:
!pip install openai==0.28.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


Install Datasets

In [2]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [3]:
!pip install datasets pandas

  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Import Libraries and Set Up API Key

In [4]:
import openai
import os
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# 파일에서 API 키 읽기
with open('/content/drive/My Drive/2024_02_ML/openai_key.txt', 'r') as file:
    api_key = file.read().strip()

# OpenAI API 키 설정
openai.api_key = api_key

# 모델 설정
MODEL = "gpt-3.5-turbo"

# API 호출 테스트
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Can you explain why this error occurs?"}
    ]
)

# 출력 결과 확인
print(response['choices'][0]['message']['content'])


Mounted at /content/drive
Of course! In order to assist you better, I'll need some additional information. Could you please provide me with the specific error message you are encountering, as well as any relevant code or context?


Example for using GPT at Colab

For Korean datasets

In [6]:
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "한국어로 설명해주세요: Machine Learning이 무엇입니까?"}
    ]
)

# 출력 결과 확인
print(response['choices'][0]['message']['content'])

Machine Learning은 컴퓨터가 데이터에서 패턴을 학습하고 예측을 만들어내는 인공지능의 하위 분야입니다. 컴퓨터가 프로그래밍 없이도 데이터로부터 학습할 수 있도록 알고리즘과 모델을 사용하여 작동합니다. 이러한 기술은 주로 패턴 인식, 예측 분석, 데이터 처리 등 다양한 분야에 활용됩니다.


# 1. Preparation

PubMedQA Dataset

In [13]:
from datasets import load_dataset

# PubMedQA 데이터셋 로드
pubmedqa = load_dataset("pubmed_qa", "pqa_labeled")

# Train 데이터만 존재함
print("Available splits:", pubmedqa.keys())

# Train 데이터를 Validation과 Test로 나누기
dataset = pubmedqa["train"].train_test_split(test_size=0.2, seed=42)  # 80% Train, 20% Test
train_validation = dataset["train"].train_test_split(test_size=0.25, seed=42)  # 75% Train, 25% Validation

# 나눈 데이터 확인
train_data = train_validation["train"]  # 약 60% (전체의 75% of 80%)
validation_data = train_validation["test"]  # 약 20% (전체의 25% of 80%)
test_data = dataset["test"]  # 약 20% (전체의 20%)

print("Train size:", len(train_data))
print("Validation size:", len(validation_data))
print("Test size:", len(test_data))

Available splits: dict_keys(['train'])
Train size: 600
Validation size: 200
Test size: 200


In [14]:
# 데이터 형태 확인
print(train_data[0])

{'pubid': 25277731, 'question': 'Sternal fracture in growing children : A rare and often overlooked fracture?', 'context': {'contexts': ['Sternal fractures in childhood are rare. The aim of the study was to investigate the accident mechanism, the detection of radiological and sonographical criteria and consideration of associated injuries.', 'In the period from January 2010 to December 2012 all inpatients and outpatients with sternal fractures were recorded according to the documentation.', 'A total of 4 children aged 5-14\xa0years with a sternal fracture were treated in 2\xa0years, 2\xa0children were hospitalized for pain management and 2 remained in outpatient care.'], 'labels': ['BACKGROUND', 'METHOD', 'RESULTS'], 'meshes': ['Adolescent', 'Chest Pain', 'Child', 'Child, Preschool', 'Diagnosis, Differential', 'Fractures, Bone', 'Humans', 'Male', 'Rare Diseases', 'Sternum'], 'reasoning_required_pred': ['m', 'a', 'y', 'b', 'e'], 'reasoning_free_pred': ['m', 'a', 'y', 'b', 'e']}, 'long_a

LexGlue Datasset

In [17]:
# LexGLUE 데이터셋 로드
lexglue = load_dataset("lex_glue", "ecthr_a")

# 데이터 확인
print(lexglue["train"].features)
print(lexglue["train"][0])

# 데이터프레임으로 변환
lexglue_train = lexglue["train"].to_pandas()
lexglue_validation = lexglue["validation"].to_pandas()
lexglue_test = lexglue["test"].to_pandas()

# 필요한 열만 선택 (예: 텍스트, 라벨)
lexglue_train = lexglue_train[["text", "labels"]]
print(lexglue_train.head())


{'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'labels': Sequence(feature=ClassLabel(names=['2', '3', '5', '6', '8', '9', '10', '11', '14', 'P1-1'], id=None), length=-1, id=None)}
{'text': ['11.  At the beginning of the events relevant to the application, K. had a daughter, P., and a son, M., born in 1986 and 1988 respectively. P.’s father is X and M.’s father is V. From March to May 1989 K. was voluntarily hospitalised for about three months, having been diagnosed as suffering from schizophrenia. From August to November 1989 and from December 1989 to March 1990, she was again hospitalised for periods of about three months on account of this illness. In 1991 she was hospitalised for less than a week, diagnosed as suffering from an atypical and undefinable psychosis. It appears that social welfare and health authorities have been in contact with the family since 1989.', '12.  The applicants initially cohabited from the summer of 1991 to July 1993. In 1991

In [16]:
print(lexglue_train.columns)

Index(['text', 'labels'], dtype='object')
